<a href="https://colab.research.google.com/github/SSRaylia/Optimization/blob/master/Optimization_Problem_with_Binary_Choice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background

LQM is a middle-sized hotel chain that is considering expanding to more locations. LQM used data on 75 existing inn locations to build a linear regression model to predict “Profitability”, computed at the operating margin, or earnings before interest and taxes divided by total revenue. They tried many independent variables and came up with the final model

\begin{equation}
Profitability = 39.05 - 5.41 \times State Population per Inn + 5.86 \times Price of the Inn - 3.09 \times Square Root of the Median Income in the Area + 1.75 \times College Students in the Area
\end{equation}

All independent variables are significant and were normalized to have mean zero and standard deviation 

# A. Calculating Predicted Profitability for Each Hotel

In [0]:
import pandas as pd

In [0]:
hotel = pd.read_csv('https://raw.githubusercontent.com/SSRaylia/homework/master/SelectingHotels%20MSBA.csv')

In [0]:
hotel.head()

,Hotel,Location,Price,Price (normalized),Square Root of Median Income (normalized),College Students in Area (normalized),State Population Per Inn (normalized),Predicted Profitability
0,1,"Eureka, California",2925000,-0.30,-0.81,-0.54,-1.00,44.24
1,2,"Fresno, California",10000000,1.70,-0.41,0.31,-0.47,53.38
2,3,"Fresno, California",3750000,-0.07,-0.41,0.31,-0.47,43.02
3,4,"Fresno, California",3500000,-0.14,-0.41,0.31,-0.47,42.61
4,5,"Fresno, California",325000,-1.04,-0.41,0.31,-0.47,37.34


In [0]:
Predicted_Profitability = 39.05 - 5.41*hotel["State Population Per Inn (normalized)"] +5.86*hotel["Price (normalized)"]-3.09*hotel["Square Root of Median Income (normalized)"] +1.75*hotel["College Students in Area (normalized)"]

In [0]:
hotel["Predicted_Profitability"] = Predicted_Profitability

In [0]:
hotel.head()

,Hotel,Location,Price,Price (normalized),Square Root of Median Income (normalized),College Students in Area (normalized),State Population Per Inn (normalized),Predicted Profitability,Predicted_Profitability
0,1,"Eureka, California",2925000,-0.30,-0.81,-0.54,-1.00,44.24,44.2599
1,2,"Fresno, California",10000000,1.70,-0.41,0.31,-0.47,53.38,53.3641
2,3,"Fresno, California",3750000,-0.07,-0.41,0.31,-0.47,43.02,42.9919
3,4,"Fresno, California",3500000,-0.14,-0.41,0.31,-0.47,42.61,42.5817
4,5,"Fresno, California",325000,-1.04,-0.41,0.31,-0.47,37.34,37.3077


# B. Build an Optimization Model to Select Hotels to Buy With the Budget Given.

**Assume we have a budget of 10 Million.**

The objective is to maximize predicted profitability with constraint in budget. Denote xi as hotel i, ci as the predicted profitability of each hotel, pi as the price for each hotel.

\begin{equation}
Max\quad\sum_{i=1}^{16} c_{i} x_{i}   \quad\forall i\in\{1,...,16\}\\
\end{equation}
s.t.
\begin{equation}
\sum_{i=1}^{16}   p_{i}x_{i} \leq 10000000 \quad\forall i\in\{1,...,16\}\\
\end{equation}
\begin{equation}
x_{i}=\{0,1\}
\end{equation}

In [0]:
import cvxpy as cvx
from cvxpy import *

x=cvx.Variable(16,boolean=True)

objective = cvx.Maximize(sum(cvx.multiply(hotel["Predicted_Profitability"],x))) 
c1= (2.925*x[0]+10*x[1]+3.75*x[2]+3.5*x[3]+0.325*x[4]+8.95*x[5]+1.95*x[6]+1.75*x[7]+4.9*x[8]
     +1.65*x[9]+1.125*x[10]+2.5*x[11]+1.975*x[12]+3.75*x[13]+1.475*x[14]+0.75*x[15] <= 10)
con = [c1]
prob = cvx.Problem(objective, con)
result = prob.solve()
x.value
np.round(x.value)

array([ 0., -0.,  0.,  0.,  1., -0.,  0.,  0., -0.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.])

In [0]:
result

270.0497002080151

**Thus, we will buy hotel no.5, 10,11,12,13,15, and 16, the total predicted profitability will be 270.05.**

# C. Adding Constrains: Maximum Number of Hotels Purchased in Any City to at Most 2

**New Optimization Model**

The objective is to maximize predicted profitability with constraint in budget. Denote xi as hotel i, ci as the predicted profitability of each hotel, pi as the price for each hotel.

\begin{equation}
Max\quad\sum_{i=1}^{16} c_{i} x_{i}   \quad\forall i\in\{1,...,16\}
\end{equation}
s.t.
\begin{equation}
\sum_{i=1}^{16}   p_{i}x_{i} \leq 10000000 \quad\forall i\in\{1,...,16\}\\
\sum_{i=1}^5 x_{i}\leq 2 \quad\forall i\in\{1,...,5\}\\
\sum_{i=7}^9 x_{i}\leq 2 \quad\forall i\in\{7, 8, 9\}\\
\sum_{i=10}^{16} x_{i}\leq 2\quad\forall i\in\{10,...,16\}\\
\end{equation}
\begin{equation}
x_{i}=\{0,1\}
\end{equation}

In [0]:
x=cvx.Variable(16,boolean=True)

objective = cvx.Maximize(sum(cvx.multiply(hotel["Predicted_Profitability"],x))) 
c1= (2.925*x[0]+10*x[1]+3.75*x[2]+3.5*x[3]+0.325*x[4]+8.95*x[5]+1.95*x[6]+1.75*x[7]+4.9*x[8]
     +1.65*x[9]+1.125*x[10]+2.5*x[11]+1.975*x[12]+3.75*x[13]+1.475*x[14]+0.75*x[15] <= 10)
c2= (x[0]+x[1]+x[2]+x[3]+x[4] <=2)
c3= (x[6]+x[7]+x[8] <=2)
c4= (x[9]+x[10]+x[11]+x[12]+x[13]+x[14]+x[15] <=2)
con = [c1,c2,c3,c4]
prob = cvx.Problem(objective, con)
result = prob.solve()
x.value
np.round(x.value)

array([1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.])

In [0]:
result

205.71680004527826

**Now we will only buy hotel no.1, 5, 7,8,10, and 11, and the optimal objective function value is 205.7, which is lower than before.**